In [42]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [43]:
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")
INDEX_NAME = os.getenv("INDEX_NAME")
INDEX_ID = os.getenv("INDEX_ID")
DB_USER= os.getenv("CLOUD_SQL_USER1")
DB_PASS = os.getenv("CLOUD_SQL_PASSWORD")
GCS_BUCKET_NAME = os.getenv("GCS_BUCKET_NAME")
DOC_PROCESSOR_NAME = os.getenv("DOC_PROCESSOR_NAME")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_community import GCSFileLoader, GCSDirectoryLoader

def load_pdf(file_path):
    return PyPDFLoader(file_path)

loader = GCSDirectoryLoader(project_name=PROJECT_ID,
                       bucket=GCS_BUCKET_NAME,
                       prefix="data/pdfs/",
                       loader_func=load_pdf)
documents = loader.load()

In [33]:
documents[:3]

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2021-12-01T15:46:35+00:00', 'moddate': '2021-12-01T15:46:35+00:00', 'source': 'gs://langchain-book-test-bucket/data/pdfs/Field_Trip_FrontPage[1]-converted.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='A \nGEOPHYSICAL FIELD WORK REPORT \nCARRIED OUT IN IGARRA, AKOKO-EDO LOCAL GOVERNMENT, \nEDO STATE, NIGERIA. \n \nBY \nIBRAHIM OLAWALE O. \nAGP/15/1930 \n \nSUBMITTED TO \nTHE DEPARTMENT OF APPLIED GEOPHYSICS, \nSCHOOL OF EARTH AND MINERAL SCIENCES, \nTHE FEDERAL UNIVERSITY OF TECHNOLOGY AKURE, ONDO STATE, NIGERIA. \nIN PARTIAL FULFILLMENT OF THE REQUIREMENT FOR THE AWARD OF \nBACHELOR OF TECHNOLOGY (B.TECH) IN APPLIED GEOPHYSICS. \n \n \n          MARCH, 2019'),
 Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2021-12-01T15:46:35+00:00', 'moddate': 

### Chunking

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200)

In [37]:
texts = text_splitter.split_documents(documents)
len(texts), type(texts[0])

(724, langchain_core.documents.base.Document)

In [48]:
from langchain_google_community import DocAIParser
from langchain_core.document_loaders.blob_loaders import Blob

parser = DocAIParser(location="us",
                     processor_name=DOC_PROCESSOR_NAME,
                     gcs_output_path="gs://{}/data/output/".format(GCS_BUCKET_NAME))

In [53]:
inp_path = "gs://{}/data/pdfs/Full Chapters Thesis.pdf".format(GCS_BUCKET_NAME)
blob = Blob(path=inp_path)
docs = list(parser.lazy_parse(blob))

In [55]:
len(docs)

97

In [56]:
operations = parser.docai_parse([blob])

In [58]:
import time
while parser.is_running(operations):
    time.sleep(0.5)
results = parser.get_results(operations)

In [59]:
results

[DocAIParsingResults(source_path='gs://langchain-book-test-bucket/data/pdfs/Full Chapters Thesis.pdf', parsed_path='gs://langchain-book-test-bucket/data/output/10754326321655577915/0')]

In [60]:
docs = list(parser.parse_from_results(results))

In [62]:
len(docs)

97